In [1]:
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import wandb
from utils import *
from model import *
from dataset import *
from tqdm import tqdm
from torch.utils.data import DataLoader

# Vocab 구축

In [2]:
#%pip install sentencepiece

In [3]:
import sentencepiece as spm

model_file = "./data/kowiki.model" 

# SentencePiece 모델 로드
sp = spm.SentencePieceProcessor()
sp.load(model_file)


True

# 학습 데이터 불러오기

In [4]:
import pandas as pd
dataset_name = 'ChatbotData'
csv_path = './data/ChatbotData.csv'
df = pd.read_csv(csv_path)
df.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [5]:
#시퀀스 max 길이 찾기
seq_max_len = 0
for line in list(df['Q'].values):
    leng = len(sp.encode_as_ids(line))
    if seq_max_len < leng:
        seq_max_len = leng
print("Q seq_max_len:", seq_max_len)


for line in list(df['A'].values):
    leng = len(sp.encode_as_ids(line))
    if seq_max_len < leng:
        seq_max_len = leng
print("A seq_max_len:", seq_max_len)

#학습데이터 Vocab 적용해보기
ids_stack = []
for line in list(df['Q'][:5].values):
    pieces = sp.encode_as_pieces(line)
    ids = sp.encode_as_ids(line)
    ids += (seq_max_len-len(ids))*[0]
    ids_stack.append(ids)
    print("Original Text:", line)
    print("Tokens:", pieces)
    print("IDs:", ids)
    print()


Q seq_max_len: 33
A seq_max_len: 43
Original Text: 12시 땡!
Tokens: ['▁12', '시', '▁', '땡', '!']
IDs: [196, 3613, 3587, 6308, 4232, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Original Text: 1지망 학교 떨어졌어
Tokens: ['▁1', '지', '망', '▁학교', '▁떨어', '졌', '어']
IDs: [7, 3601, 3946, 1176, 1522, 3940, 3624, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Original Text: 3박4일 놀러가고 싶다
Tokens: ['▁3', '박', '4', '일', '▁놀', '러', '가', '고', '▁싶', '다']
IDs: [49, 3905, 3656, 3620, 2356, 3716, 3599, 3600, 2636, 3589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Original Text: 3박4일 정도 놀러가고 싶다
Tokens: ['▁3', '박', '4', '일', '▁정도', '▁놀', '러', '가', '고', '▁싶', '다']
IDs: [49, 3905, 3656, 3620, 1121, 2356, 3716, 3599, 3600, 2636, 3589, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Origi

In [6]:
train_dataset = ChatBotDataset(csv_path, seq_max_len)
val_dataset = ChatBotDataset(csv_path, seq_max_len, train=False)

print("train_dataset length:", len(train_dataset))
print("val_dataset length:", len(val_dataset)) 

train_dataset length: 10640
val_dataset length: 1183


# Scaled_dot_Attention 계산 확인

In [7]:
n_dim = 3
q = torch.tensor([[[1,2,3],
                   [3,2,1],
                   [4,5,6]],
                  
                  [[3,2,2],
                   [1,1,1],
                   [5,2,4]]], dtype=torch.float32)
k = q.transpose(-1,-2)
token_ids = torch.tensor([[1,1,0], [1,0,0]])


scaled_attention = torch.matmul(q,k) / np.sqrt(n_dim)
masked_attention =  making_padding_mask(scaled_attention, token_ids)
attention_score = torch.softmax(masked_attention, dim=-1)
output = torch.matmul(attention_score, q)
print("정답:", output)


정답: tensor([[[1.1807, 2.0000, 2.8193],
         [2.8193, 2.0000, 1.1807],
         [1.1807, 2.0000, 2.8193]],

        [[3.0000, 2.0000, 2.0000],
         [3.0000, 2.0000, 2.0000],
         [3.0000, 2.0000, 2.0000]]])


In [8]:
#확인할 때 q,k,v의 가중치는 제외하고 확인해볼 것
model = ScaledDotProductAttention(3)
model(q,q,q,token_ids)

tensor([[[-0.4208,  0.8233,  0.1114],
         [-0.4719,  0.8608,  0.1540],
         [-0.5877,  0.9457,  0.2505]],

        [[ 0.4391, -0.1292, -0.8138],
         [ 0.4391, -0.1292, -0.8138],
         [ 0.4391, -0.1292, -0.8138]]], grad_fn=<UnsafeViewBackward0>)

# Hyper Parameter

In [9]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

#model hyperparameter
model_name = 'transformer_decoderloop'
n_seq = seq_max_len
n_vocab = sp.vocab_size()
# 논문의 절반으로 setting
n_dim = 256
n_head = 4
n_layer = 2

# dataloader hyperparameter
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#train, Loss, Optimizer hyperparameter
epochs = 100
lr = 1e-4

# Transformer Model

In [10]:
model = Transformer(n_seq, n_vocab, n_dim, n_head, n_layer, device=device).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [11]:
def train(loss_fn, optimizer, save_path, eval_step=1,resume=False, run_id=None):
    
    wandb.init(
        project='Transformer_ChatBot',
        name = f'{model_name}_{dataset_name}',
        resume="allow" if resume else None,
        id=run_id if resume else None,
        config={
            "architecture": model_name,
            "dataset": dataset_name,
            "batch_size": batch_size,
            "lr": lr,
            "epochs": epochs,
            "n_seq": n_seq,
            "n_vocab": n_vocab,
            "n_dim": n_dim,
            "n_head": n_head,
            "n_layer": n_layer,
            "loss_fn": "CrossEntropyLoss",
            "optimizer": "Adam",
        })
    
    softmax = nn.Softmax(dim=-1)
    model.train()
    best_val_loss = np.inf
    # BOS 토큰 ID를 가져옵니다.
    bos_token_id = sp.bos_id()
    for epoch in tqdm(range(epochs), ascii=True, desc="epoch"):
        print()
        print(f"********** epoch{epoch+1} train start **********")
        train_loss = 0
        for idx, (q, a) in enumerate(train_dataloader):
            q, a = q.to(device), a.to(device)
            # a를 bos_a로 변경 (오른쪽 시프트)
            bos_a = shift_right(a, bos_token_id)
            
            # 디코더 루프별로 손실 계산
            total_loss = 0.0
            pred_lst = []
            # 문장 만들 index
            indx = np.random.randint(0, len(q))  # 0 ~ batch_size 사이의 숫자 랜덤으로 뽑기
            for i in range(n_seq):
                # 디코더 입력 업데이트
                current_dec_input = torch.cat([bos_a[:, :i + 1], torch.zeros((bos_a.size(0), n_seq - i - 1), device=device, dtype=torch.long)], dim=1)
            
                # 모델 예측
                output = model(q, current_dec_input)
            
                # 다음 토큰 예측
                pred = output[:, i, :]  # 현재 스텝의 출력을 사용
                target = a[:, i]  # 현재 스텝의 목표 토큰
                pred_lst.append(softmax(pred[indx]).argmax(dim=-1).item())
                # 손실 계산
                loss = loss_fn(pred, target)
                total_loss += loss
                
            avg_loss = total_loss / n_seq
            train_loss += avg_loss.item()
        
            # 역전파 및 최적화
            optimizer.zero_grad()
            avg_loss.backward()
            optimizer.step()
            
            if idx % (len(train_dataloader)//3) == 0:
                print("epoch: {}, loss: {}".format(epoch+1, avg_loss.item()))
                # 문장 만들기
                pred_sentc = sp.DecodeIds(pred_lst)
                label_sentc = sp.DecodeIds(a[indx].tolist())
                q_sentc = sp.DecodeIds(q[indx].tolist())
                print("Q:", q_sentc)
                print("Pred:", pred_sentc)
                print("Label:", label_sentc)  
        
        print("epoch: {}, train_loss: {}".format(epoch+1, train_loss/len(train_dataloader)))
        print("********** train end **********")
        
        #wandb에 로그 기록
        train_metrics = {'train_loss': train_loss/len(train_dataloader)}
        wandb.log(train_metrics, step=epoch)
        print()
        if epoch % eval_step == 0:
            with torch.no_grad():
                print("********** eval start **********")
                model.eval()
                val_loss = 0.0
                for idx, (q, a) in enumerate(val_dataloader):
                    q, a = q.to(device), a.to(device)
                    bos_a = shift_right(a, bos_token_id)
                    # 디코더 루프별로 손실 계산
                    total_loss = 0.0
                    pred_lst = []
                    # 문장 만들 index
                    indx = np.random.randint(0, len(q))  # 0 ~ len(val_dataset) 사이의 숫자 랜덤으로 뽑기
                    for i in range(n_seq):
                        # 디코더 입력 업데이트
                        current_dec_input = torch.cat([bos_a[:, :i + 1], torch.zeros((bos_a.size(0), n_seq - i - 1), device=device, dtype=torch.long)], dim=1)
                    
                        # 모델 예측
                        output = model(q, current_dec_input)
                    
                        # 다음 토큰 예측
                        pred = output[:, i, :]  # 현재 스텝의 출력을 사용
                        target = a[:, i]  # 현재 스텝의 목표 토큰
                        pred_lst.append(softmax(pred[indx]).argmax(dim=-1).item())
                        # 손실 계산
                        loss = loss_fn(pred, target)
                        total_loss += loss
                    
                    avg_loss = total_loss / n_seq
                    val_loss += avg_loss.item()
               
                # 문장 만들기
                pred_sentc = sp.DecodeIds(pred_lst)
                label_sentc = sp.DecodeIds(a[indx].tolist())
                q_sentc = sp.DecodeIds(q[indx].tolist())
                print("Q:", q_sentc)
                print("Pred:", pred_sentc)
                print("Label:", label_sentc)  
               
                avg_val_loss = val_loss / len(val_dataloader)
                if best_val_loss > avg_val_loss:
                    best_val_loss = avg_val_loss
                    torch.save(model.state_dict(), save_path)
                    print("\n****best model saved****\n")
                
                print("epoch: {}, val_loss: {}".format(epoch + 1, avg_val_loss))
                val_metrics = {'val_loss': avg_val_loss}
                wandb.log(val_metrics, step=epoch)
              
    wandb.finish()

In [12]:
save_path = f'./weight/best_{model_name}.pt'
train(loss_fn, optimizer,save_path, eval_step=1)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dwkim8155 (boostcamp-oif). Use `wandb login --relogin` to force relogin


epoch:   0%|          | 0/100 [00:00<?, ?it/s]


********** epoch1 train start **********
epoch: 1, loss: 9.61805534362793
Q: 친구한테 쌓인 게 안 풀려
Pred: 들을 레 배경敗딪 만나ザ라스기의泳 드라마吏뎀峰폴 즐 지도 지도 원칙 원칙 원칙봐봐봐봐봐位 지도 지도 지도 지도 지도 지도隣隣隣隣휴휴휴휴휴휴
Label: 대화하면서 얽힌 감정들을 풀어보세요.
epoch: 1, loss: 1.9782787561416626
Q: 다 알고 있을까?
Pred: 
Label: 다 알 수는 없어요.
epoch: 1, loss: 1.5723509788513184
Q: 엄마랑 둘이 여행가
Pred: 요요요.
Label: 좋은 여행이 될 거예요.
epoch: 1, loss: 1.4829152822494507
Q: 세상에서 제일 쓸데없는 게 짝사랑 같아.
Pred: 세 요요요요요.
Label: 감정소모가 심하긴 해요.
epoch: 1, train_loss: 2.131368714178394
********** train end **********

********** eval start **********


epoch:   1%|1         | 1/100 [08:29<14:00:50, 509.60s/it]

Q: 1년 넘게 만났는데 지금도 불타올라
Pred: 이 요요요요요요.
Label: 정열적인 사랑을 하고 있나봐요.

****best model saved****

epoch: 1, val_loss: 1.3439699850584332

********** epoch2 train start **********
epoch: 2, loss: 1.4544585943222046
Q: 알바하는데 같이 일하는 사람 좋아져
Pred: 도세세요요요요세요.
Label: 용기내서 고백해보세요.
epoch: 2, loss: 1.25917649269104
Q: 편두통 온다
Pred: 하요요요.
Label: 잠시 쉬었다 가세요.
epoch: 2, loss: 1.0620534420013428
Q: 이별을 해야하나봐
Pred: 이 해요.
Label: 마음이 문제네요.
epoch: 2, loss: 1.1690996885299683
Q: 오늘부터 짝사랑을 끝내려 합니다.
Pred: 들게해 거해요돼요요이요해요.
Label: 힘든 결정이었을텐데 맘고생 많았어요.
epoch: 2, train_loss: 1.2193384052750593
********** train end **********

********** eval start **********


...

Q: 한달 다되가네
Pred: 생각게 말. 생각덧 일 건았요.
Label: 그러게요. 어느덧 한달이에요.
epoch: 30, val_loss: 0.7228733552129645

********** epoch31 train start **********
epoch: 31, loss: 0.1502198576927185
Q: 정말 아직 너무 사랑합니다 잡고 싶습니다
Pred: 다시 잡는다면 후회할 거예요.
Label: 다시 잡는다면 후회할 거예요.
epoch: 31, loss: 0.15972517430782318
Q: 셀카 좀 잘 찍고 싶다
Pred: 연습해보세요.
Label: 연습해보세요.
epoch: 31, loss: 0.19429582357406616
Q: 영화보고 왔다
Pred: 저도 영화 보여주세요.
Label: 저도 영화 보여주세요.
epoch: 31, loss: 0.22896885871887207
Q: 썸 끝나고 잡아도 될까
Pred: 좋아은 언제 타 상 시작해도 늦니다.
Label: 사랑은 언제든 다시 시작해도 됩니다.
epoch: 31, train_loss: 0.20631861213795438
********** train end **********

********** eval start **********


epoch:  31%|###1      | 31/100 [5:20:22<11:09:31, 582.20s/it]

Q: 진심일까
Pred: 너무 물어보세요.
Label: 직접 물어보세요.
epoch: 31, val_loss: 0.7326420953399256

********** epoch32 train start **********
epoch: 32, loss: 0.18364588916301727
Q: 여자친구가 계획적이지 않아서 스트레스 받아.
Pred: 성격도이니 받아들여야세요.
Label: 성격 차이니 받아들여보세요.


epoch:  31%|###1      | 31/100 [5:22:31<11:57:52, 624.24s/it]


KeyboardInterrupt: 

In [22]:
#저장된 best 가중치 불러오기
model = Transformer(n_seq, n_vocab, n_dim, n_head, n_layer, device=device).to(device)
save_path = f'./weight/best_{model_name}.pt'
state_dict = torch.load(save_path)
model.load_state_dict(state_dict)

model.eval()
prompt = "잘 안되네"
output = generate_sentc(model, sp, n_seq, prompt, device)
print("입력 문장:", prompt)
print("출력 문장:", output)

입력 문장: 잘 안되네
출력 문장: 때
